In [ ]:
import tensorflow as tf 
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import random, numpy as np, os
#import tensorflow_datasets as tfds
#tfds.disable_progress_bar()
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.run_functions_eagerly(True)
def config_gpu(mp=False):
    print('Eager Model : ', tf.executing_eagerly())
    print('TensorFlow Cuda Built Test : ', tf.test.is_built_with_cuda)
    print('TensorFlow GPU Detected : ', tf.test.gpu_device_name())
    print('TensorFlow System Cuda Version : ', tf.sysconfig.get_build_info()["cuda_version"])
    print('TensorFlow System CudNN Version : ', tf.sysconfig.get_build_info()["cudnn_version"] )

    AUTO = tf.data.AUTOTUNE
    GPUS = tf.config.list_physical_devices('GPU')
    if GPUS:
        try:
            for GPU in GPUS:
                tf.config.experimental.set_memory_growth(GPU, True)
                logical_gpus = tf.config.list_logical_devices('GPU')
                print(len(GPUS), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
        except RuntimeError as  RE:
            print(RE)
    if mp:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print('Mixed precision enabled')
        
#tf.keras.utils.set_random_seed(100)
config_gpu(mp=False)

# <a id='intro'>Plan</a>
#### [Wavelet with residual block](#the_destination)
#### [Efficient-net](#Efficient_net)
#### [Ensemblelearning](#Ensemble_learning)
#### [Metrics with Graph](#graph)
#### [Grad-CAM](#Gradcam)
#### [Result Metrics](#metrics)

# <a id=''>TF Dataset and Preprocess</a>

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from keras.layers import Lambda
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile

In [ ]:
import os, cv2
import random, math
import numpy as np 
import pandas as pd
from PIL import Image
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import tensorflow, keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# metadata_df = pd.read_csv('D:\Doctorat\HAM10000/HAM10000_metadata.csv')
# metadata_df.head()

In [ ]:
train_path = '/kaggle/input/basedir/base_dir/train_dir'
test_path = '/kaggle/input/basedir/base_dir/val_dir'
batch_size = 16

train_batch_size = 4
val_batch_size = 4
image_size = 224


In [ ]:
def custom_augmentation(np_tensor):

  def random_contrast(np_tensor):
    return np.array(tf.image.random_contrast(np_tensor, 0.5, 2))

  def random_saturation(np_tensor):
    return np.array(tf.image.random_saturation(np_tensor, 0.2, 3))

  def random_crop(np_tensor):
    #cropped height between 70% to 130% of an original height
    new_height = int(np.random.uniform(0.7, 1.30) * np_tensor.shape[0])
    #cropped width between 70% to 130% of an original width
    new_width = int(np.random.uniform(0.7, 1.30) * np_tensor.shape[1])
    # resize to new height and width
    cropped = tf.image.resize_with_crop_or_pad(np_tensor, new_height, new_width)
    return np.array(tf.image.resize(cropped, np_tensor.shape[:2]))

  def gaussian_noise(np_tensor):
    mean = 0
    # variance: randomly between 1 to 25
    var = np.random.randint(1, 26)
    # sigma is square root of the variance value
    noise = np.random.normal(mean,var**0.5,np_tensor.shape)
    return np.clip(np_tensor + noise, 0, 255).astype('int')

  def cutout(np_tensor):
    cutout_height = int(np.random.uniform(0.1, 0.2) * np_tensor.shape[0])
    cutout_width = int(np.random.uniform(0.1, 0.2) * np_tensor.shape[1])
    cutout_height_point = np.random.randint(np_tensor.shape[0]-cutout_height)
    cutout_width_point = np.random.randint(np_tensor.shape[1]-cutout_width)
    np_tensor[cutout_height_point:cutout_height_point+cutout_height, cutout_width_point:cutout_width_point+cutout_width, :] = 127
    return np_tensor

  if (np.random.uniform() < 0.1):
    np_tensor = random_contrast(np_tensor)
  if (np.random.uniform() < 0.1):
    np_tensor = random_saturation(np_tensor)
  if (np.random.uniform() < 0.2):
    np_tensor = random_crop(np_tensor)
  if (np.random.uniform() < 0.2):
    np_tensor = gaussian_noise(np_tensor)
  if (np.random.uniform() < 0.3):
    np_tensor = cutout(np_tensor)
  return np.array(np_tensor)

In [ ]:

datagen = ImageDataGenerator(#preprocessing_function=custom_augmentation,
                            rescale=1./255,
                             preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
                            horizontal_flip = True,
                            vertical_flip=True
)
datagen1 = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
                             rescale=1./255,)
                             #horizontal_flip = True,
                             #vertical_flip=True,)


train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size,)
                                           #subset='training')

valid_batches = datagen.flow_from_directory(test_path,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size,)

test_batches = datagen1.flow_from_directory(test_path,
                                            target_size=(image_size,image_size),
                                            batch_size=32,
                                            shuffle=False,)
                                          #subset='validation')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Update this with your actual class names
class_names = ['class0', 'class1', 'class2', 'class3', 'class4', 'class5', 'class6']

images, labels = next(train_batches)
plt.figure(figsize=(12, 6))
for i in range(len(images)):  # This will be the number of images in the batch, which is 4 in your case
    plt.subplot(1, 4, i+1)
    plt.imshow(images[i])
    # Decode the one-hot encoded label to get the class index
    class_index = np.argmax(labels[i])
    # Use the class index to get the corresponding class name
    plt.title(f'Class: {class_names[class_index]}')
plt.axis('off')
plt.show()


In [ ]:
#densneet
%matplotlib inline
image, label = next(train_batches)
plt.figure(figsize=(20,10))
inline = 4
for i in range(inline):
    #print(image.size())
    plt.subplot(2, inline, i%inline +1)
    plt.axis('off')
    plt.imshow(image[i])
    plt.title(f'Label: {label}')
    plt.subplot(2, inline, i%inline +5)
    plt.axis('off')
    plt.imshow(image[i].astype(np.uint8))

# import matplotlib.pyplot as plt
# import numpy as np

# # Assuming you have an array or list of class names corresponding to the indices of the one-hot encoding
# class_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']  # update this with your actual class names

# %matplotlib inline
# image, label = next(train_batches)  # Assuming train_batches is an iterator that yields batches of images and one-hot labels
# plt.figure(figsize=(20,10))
# inline = 4

# for i in range(inline):
#     plt.subplot(2, inline, i%inline +1)
#     plt.axis('off')
#     plt.imshow(image[i])
#     # Decode the one-hot encoded label to get the class index
#     class_index = np.argmax(label[i])
#     # Use the class index to get the corresponding class name
#     plt.title(f'Class: {class_names[class_index]}')

#     plt.subplot(2, inline, i%inline +5)
#     plt.axis('off')
#     # Assuming image[i] is already in the correct dtype for visualization, if not, cast it appropriately
#     plt.imshow(image[i].astype(np.uint8))

# plt.show()



In [ ]:
images, labels = next(train_batches)
print('Image batch shape:', images.shape)
print('Label batch shape:', labels.shape)
print('First image label:', labels[0])


# Configuration

In [ ]:
# Setting TPU as main device for training, if you get warnings while working with tpu's ignore them.

DEVICE = 'TPU'
if DEVICE == 'TPU':
    print('connecting to TPU...')
    try:        
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print('Could not connect to TPU')
        tpu = None

    if tpu:
        try:
            print('Initializing  TPU...')
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print('TPU initialized')
        except _:
            print('Failed to initialize TPU!')
    else:
        DEVICE = 'GPU'

if DEVICE != 'TPU':
    print('Using default strategy for CPU and single GPU')
    strategy = tf.distribute.get_strategy()

if DEVICE == 'GPU':
    print('Num GPUs Available: ',
          len(tf.config.experimental.list_physical_devices('GPU')))

print('REPLICAS: ', strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
cfg = dict(
           batch_size=4,
           img_size=384,
    
           lr_start=0.000005,
           lr_max=0.00000125,
           lr_min=0.000001,
           lr_rampup=5,
           lr_sustain=0,
           lr_decay=0.8,
           epochs=12,
    
           transform_prob=1.0,
           rot=180.0,
           shr=2.0,
           hzoom=8.0,
           wzoom=8.0,
           hshift=8.0,
           wshift=8.0,
    
           optimizer='adam',
           label_smooth_fac=0.05,
           tta_steps=20
            
        )

# <a id='the_destination'>Wavelet with residual block</a>
#### [Return Contents](#intro)

# <a id='Xception'>Xception</a>
#### [Return Contents](#intro)

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for index, layer in enumerate(base_model.layers):
    print(index, layer.name)

In [ ]:
def get_model():
    
    ''' This function gets the layers inclunding efficientnet ones. '''
    
    model_input = tf.keras.Input(shape=(224,224, 3),
                                 name='img_input')

    dummy = tf.keras.layers.Lambda(lambda x: x)(model_input)



    x = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False,
                                                  input_shape=(224,224, 3),
                                                  pooling='avg')(dummy)
    x = tf.keras.layers.Dense(7, activation='softmax')(x)
    
    """
    x = efn.EfficientNetB4(include_top=False,
                           weights='noisy-student',
                           input_shape=(224,224, 3),
                           pooling='avg')(dummy)
    x = tf.keras.layers.Dense(7, activation='softmax')(x)
    outputs.append(x)
    """
    """
    x = efn.EfficientNetB5(include_top=False,
                           weights='noisy-student',
                           input_shape=(224,224, 3),
                           pooling='avg')(dummy)
    x = tf.keras.layers.Dense(7, activation='softmax')(x)
    outputs.append(x)
    """

    model = tf.keras.Model(model_input, x, name='aNetwork')
    model.summary()
    return model

In [ ]:
efficient = get_model()
for index, layer in enumerate(efficient.layers):
    print(index, layer.name)

In [ ]:
def compileNewModel():
    
    ''' Configuring the model with losses and metrics. '''    
    
    with strategy.scope():
        model = efficient

    with strategy.scope():
        model.compile(optimizer='adam', loss='categorical_crossentropy', 
                      metrics=['accuracy'])
        #tf.keras.metrics.AUC(name='auc')
    return model

def getLearnRateCallback(cfg):
    
    ''' Using callbacks for learning rate adjustments. '''
    
    lr_start = cfg['lr_start']
    lr_max = cfg['lr_max'] * strategy.num_replicas_in_sync * cfg['batch_size']
    lr_min = cfg['lr_min']
    lr_rampup = cfg['lr_rampup']
    lr_sustain = cfg['lr_sustain']
    lr_decay = cfg['lr_decay']

    def lrfn(epoch):
        if epoch < lr_rampup:
            lr = (lr_max - lr_start) / lr_rampup * epoch + lr_start
        elif epoch < lr_rampup + lr_sustain:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_rampup -
                                                lr_sustain) + lr_min
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback


def learnModel(model, ds_train, cfg, ds_val):
    
    ''' Fitting things together for training '''
    
    filepath1 = "Efficient_att_ens.h5"
    checkpoint = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)
    callbacks = [getLearnRateCallback(cfg), checkpoint]

    history = model.fit(ds_train,
                        validation_data=ds_val,
                        verbose=True,
                        epochs=120,
                        callbacks=callbacks)
    return history

In [ ]:
#efficient net bou7dou i have to save it tooo.. # hada wavelet with global dolg i have to save it (la 2eme fois je dois uploader les meilleurs poids avant enregistremnt)
model1 = compileNewModel()
history1 = learnModel(model1, train_batches, cfg, valid_batches)